<a href="https://colab.research.google.com/github/cod3astro/leetcode_and_hackerrank/blob/main/colab_leetcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Write a solution to find all dates' id with higher temperatures compared to its previous dates (yesterday).

Return the result table in any order.

The result format is in the following example.

Example 1:

Input:
Weather table:
+----+------------+-------------+
| id | recordDate | temperature |
+----+------------+-------------+
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |
+----+------------+-------------+
Output:
+----+
| id |
+----+
| 2  |
| 4  |
+----+
Explanation:
In 2015-01-02, the temperature was higher than the previous day (10 -> 25).
In 2015-01-04, the temperature was higher than the previous day (20 -> 30).

In [ ]:
import pandas as pd
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
  # Converting the date time from a string to a date time
    weather["recordDate"] = pd.to_datetime(weather["recordDate"])

  # pd.Timedelta(days=1) → Represents a time duration of 1 day
  # Subtracting it from recordDate creates a new column yesterday
  # This column represents "yesterday’s date" for each row
    weather["yesterday"] = weather["recordDate"] - pd.Timedelta(days=1)

  # It matches each day with its yesterday.
  # left_on="yesterday" → use the yesterday column from the left table
  # right_on="recordDate" → match it to the recordDate of the right table.
  # Since we’re merging the table with itself, this is a self-join.
  # suffixes=("", "_y") → keeps today’s columns as-is, and renames yesterday’s columns (like temperature_y).
    merged = weather.merge(weather, left_on="yesterday", right_on="recordDate", suffixes=("", "_y"))

  # Checks if the temperature for the present day is greater than the temperature of the previous day
    result = merged.loc[merged["temperature"] > merged["temperature_y"], ["id"]]
    return result


Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to determine the number of players who logged in on the day immediately following their initial login, and divide it by the number of total players.

The result format is in the following example.



Example 1:

Input:
Activity table:
+-----------+-----------+------------+--------------+
| player_id | device_id | event_date | games_played |
+-----------+-----------+------------+--------------+
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-03-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |
+-----------+-----------+------------+--------------+
Output:
+-----------+
| fraction  |
+-----------+
| 0.33      |
+-----------+
Explanation:
Only the player with id 1 logged back in after the first day he had logged in so the answer is 1/3 = 0.33

In [ ]:
import pandas as pd

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
  # Convert the columns from string to a datetime column
    activity['event_date'] = pd.to_datetime(activity['event_date'])
  # Group the dataframe by player_id and return the min value of each group of event_date col, then assign it to a new var
    first_login = activity.groupby('player_id')['event_date'].min().reset_index()
  # Rename event_date column
    first_login = first_login.rename(columns={'event_date': 'first_login'})
  # Merge activity column with the first_login on the column 'player_id'
    df = activity.merge(first_login, on='player_id')
  # Create a new column which is a day + the next_day column
    df['next_day'] = df['first_login'] + pd.Timedelta(days=1)
  # Create a new boolean column named logged_next_day
    df['logged_next_day'] = df['event_date'] == df['next_day']
  # Group the dataframe by the new boolean column logged_next_day
    players_logged_next_day = df.groupby('player_id')['logged_next_day'].any().sum()
  # Get the unique values of the column "player_id" in the dataframe
    total_players = df['player_id'].nunique()
    fraction = round(players_logged_next_day / total_players, 2)
    return pd.DataFrame({'fraction': [fraction]})